### WIDE RESNET

In [ ]:
!pip install tensorflow
!pip install keras_preprocessing
!pip install keras
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, Flatten, Dense, Add
from sklearn.preprocessing import LabelEncoder
import pickle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 629.3 kB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Function to create DataFrame containing image paths and labels
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

# Function to extract features from images
def extract_features(images):
    features = []
    for image in images:
        img = load_img(image, grayscale=True, target_size=(48, 48))  # Resizing images to a consistent size
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

# Define directories for train and test data
TRAIN_DIR = 'drive/MyDrive/images/train'
TEST_DIR = 'drive/MyDrive/images/test'


In [ ]:
import random

# Function to create DataFrame containing limited number of image paths and labels
def create_limited_dataframe(dir, max_images_per_label):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        label_images = [os.path.join(dir, label, imagename) for imagename in os.listdir(os.path.join(dir, label))]
        random.shuffle(label_images) # Shuffle the images for randomness
        label_images = label_images[:max_images_per_label] # Select a limited number of images per label
        image_paths.extend(label_images)
        labels.extend([label] * len(label_images))
        print(f"{label}: {len(label_images)} images completed")
    return image_paths, labels

# Define the maximum number of images per label
MAX_IMAGES_PER_LABEL = 1000

# Create limited DataFrames for train and test data
train = pd.DataFrame()
train['image'], train['label'] = create_limited_dataframe(TRAIN_DIR, MAX_IMAGES_PER_LABEL)

test = pd.DataFrame()
test['image'], test['label'] = create_limited_dataframe(TEST_DIR, MAX_IMAGES_PER_LABEL)


neutral: 1000 images completed
sad: 1000 images completed
happy: 1000 images completed
surprise: 1000 images completed
angry: 1000 images completed
angry: 960 images completed
happy: 1000 images completed
surprise: 797 images completed
neutral: 1000 images completed
sad: 1000 images completed


In [ ]:
#train = pd.DataFrame()
#train['image'], train['label'] = createdataframe(TRAIN_DIR)


In [ ]:
#test = pd.DataFrame()
#test['image'], test['label'] = createdataframe(TEST_DIR)

In [ ]:
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

# Normalize features
x_train = train_features / 255.0
x_test = test_features / 255.0

# Encode labels
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)


/usr/local/lib/python3.10/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [ ]:
def wide_resnet(input_shape, depth, width, num_classes):
    n = (depth - 4) // 6
    k = width

    def conv_bn_relu(x, filters, kernel_size, strides=(1, 1)):  # Modify the function signature
      x = Conv2D(filters=filters, kernel_size=kernel_size, padding='same', strides=strides)(x)  # Include strides
      x = BatchNormalization()(x)
      x = Activation('relu')(x)
      return x

    def residual_block(x, filters, strides=(1, 1)):
      shortcut = x
      x = conv_bn_relu(x, filters, (3, 3), strides=strides)  # Add strides argument
      x = conv_bn_relu(x, filters, (3, 3))
      if strides != (1, 1) or shortcut.shape[3] != filters:
          shortcut = Conv2D(filters, (1, 1), strides=strides, padding='same')(shortcut)
      x = Add()([x, shortcut])
      return x

    input = Input(shape=input_shape)
    x = conv_bn_relu(input, 16, (3, 3))
    for i in range(n):
        x = residual_block(x, 16 * k)
    x = residual_block(x, 32 * k, strides=(2, 2))
    for i in range(1, n):
        x = residual_block(x, 32 * k)
    x = residual_block(x, 64 * k, strides=(2, 2))
    for i in range(1, n):
        x = residual_block(x, 64 * k)
    x = AveragePooling2D(pool_size=(8, 8))(x)
    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input, outputs=x)
    return model

In [ ]:


input_shape = (48, 48, 1)

model = wide_resnet(input_shape=input_shape, depth=28, width=10, num_classes=7)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x=x_train, y=y_train, batch_size=128, epochs=50, validation_data=(x_test, y_test))

model.save("emotiondetector_wideresnet.h5")

model = load_model("emotiondetector_wideresnet.h5")

pickle.dump(model,open('/content/drive/MyDrive/comparison_model_model/trained_model_wideres', 'wb'))


Epoch 1/50
40/40 [==============================] - 9541s 239s/step - loss: 2.7665 - accuracy: 0.2240 - val_loss: 754.0695 - val_accuracy: 0.2033
Epoch 2/50
24/40 [=================>............] - ETA: 52:04 - loss: 1.6626 - accuracy: 0.2529

In [ ]:
# Define labels
label = ['angry', 'happy', 'neutral', 'sad', 'surprise']

# Function to extract features from image
def ef(image):
    img = load_img(image, grayscale=True, target_size=(48, 48))  # Resizing images to a consistent size
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0


# Prediction and visualization
image = 'drive/MyDrive/images/test/sad/22977.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[np.argmax(pred)]
print("model prediction is ", pred_label)

# Display the image
plt.imshow(img.reshape(48, 48), cmap='gray')

In [ ]:
# Define labels
label = ['angry', 'happy', 'neutral', 'sad', 'surprise']

# Function to extract features from image
def ef(image):
    img = load_img(image, grayscale=True, target_size=(48, 48))  # Resizing images to a consistent size
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0


# Prediction and visualization
image = 'drive/MyDrive/images/test/neutral/34037.jpg'
print("original image is of neutral")
img = ef(image)
pred = model.predict(img)
pred_label = label[np.argmax(pred)]
print("model prediction is ", pred_label)

# Display the image
plt.imshow(img.reshape(48, 48), cmap='gray')

In [ ]:
# Define labels
label = ['angry', 'happy', 'neutral', 'sad', 'surprise']

# Function to extract features from image
def ef(image):
    img = load_img(image, grayscale=True, target_size=(48, 48))  # Resizing images to a consistent size
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)
    return feature / 255.0


# Prediction and visualization
image = 'drive/MyDrive/images/test/surprise/14968.jpg'
print("original image is of surprise")
img = ef(image)
pred = model.predict(img)
pred_label = label[np.argmax(pred)]
print("model prediction is ", pred_label)

# Display the image
plt.imshow(img.reshape(48, 48), cmap='gray')